In [1]:
import random
import time

import scipy.io
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
import numpy as np
import scipy.io
from scipy import interp
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from interpret import show
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [2]:
def balance_dt(data, label, seed=None):
    random.seed(seed)
    ones = []
    for i in range(len(label)):
        if label[i] == 1:
            ones.append(i)
    zeros = []
    for i in range(len(label)):
        if label[i] == 0:
            zeros.append(i)
    zeros = random.sample(zeros, len(ones))
    indices = zeros + ones
    X = data[indices]
    y = label[indices]
    return X, y

In [3]:
def balance_reversed(data, label, seed=None):
    random.seed(seed)
    zeros = []
    for i in range(len(label)):
        if label[i] == 0:
            zeros.append(i)
    ones = []
    for i in range(len(label)):
        if label[i] == 1:
            ones.append(i)
    ones = random.sample(ones, len(zeros))
    indices = zeros + ones
    X = data[indices]
    y = label[indices]
    return X, y


In [4]:
def eval_with_kfold(best_clf, x, y, org_dt, org_lb):
    cros_res = cross_val_score(best_clf, x, y, cv=10, scoring='accuracy')
    print("balanced accuracy", np.mean(cros_res))
    cros_res = cross_val_score(best_clf, org_dt, org_lb, cv=10, scoring='accuracy')
    print("unbalanced accuracy", np.mean(cros_res))

    cros_res = cross_val_score(best_clf, x, y, cv=10, scoring='precision')
    print("balanced precision", np.mean(cros_res))
    cros_res = cross_val_score(best_clf, org_dt, org_lb, cv=10, scoring='precision')
    print("unbalanced precision", np.mean(cros_res))

    cros_res = cross_val_score(best_clf, x, y, cv=10, scoring='recall')
    print("balanced recall", np.mean(cros_res))
    cros_res = cross_val_score(best_clf, org_dt, org_lb, cv=10, scoring='recall')
    print("unbalanced recall", np.mean(cros_res))

    cros_res = cross_val_score(best_clf, x, y, cv=10, scoring='f1')
    print("balanced f1", np.mean(cros_res))
    cros_res = cross_val_score(best_clf, org_dt, org_lb, cv=10, scoring='f1')
    print("unbalanced f1", np.mean(cros_res))

    cros_res = cross_val_score(best_clf, x, y, cv=10, scoring='roc_auc')
    print("balanced auc", np.mean(cros_res))
    cros_res = cross_val_score(best_clf, org_dt, org_lb, cv=10, scoring='roc_auc')
    print("unbalanced auc", np.mean(cros_res))

In [5]:
mat = scipy.io.loadmat('data.mat')
org_dat = mat['OriginalData']
stand_dat = mat['Scaled_Standardization']
minmax_dat = mat['Scaled_Min_Max']
label = mat['label'][0]

In [6]:
best_sc = 0
best_x = []
best_y = []
best_es = None

SVM

In [7]:
for i in range(5):
    random.seed(i)
    X, y = balance_dt(minmax_dat, label, seed=i)

    parameters = {'kernel': ['linear', 'poly', 'rbf'], 'random_state': [i], 'C': [1, 2, 3, 4, 5]}

    clf = GridSearchCV(SVC(probability=True), parameters, n_jobs=-1, cv=10, verbose=1, scoring='f1')
    clf.fit(X, y)

    if clf.best_score_ > best_sc:
        best_sc = clf.best_score_
        best_es = clf.best_estimator_
        best_x = X
        best_y = y

Fitting 10 folds for each of 15 candidates, totalling 150 fits
Fitting 10 folds for each of 15 candidates, totalling 150 fits
Fitting 10 folds for each of 15 candidates, totalling 150 fits
Fitting 10 folds for each of 15 candidates, totalling 150 fits
Fitting 10 folds for each of 15 candidates, totalling 150 fits


In [8]:
print("-----------------Results--------------------")
print("Best score: ", best_sc)
print(best_es)

-----------------Results--------------------
Best score:  0.5991735897279613
SVC(C=5, probability=True, random_state=2)


In [9]:
eval_with_kfold(best_es, best_x, best_y, minmax_dat, label)

balanced accuracy 0.6886952157912345
unbalanced accuracy 0.7622667615433271
balanced precision 0.7059181851388352


/Users/nayanayeshlur/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


unbalanced precision 0.6553828931606681
balanced recall 0.5443755831000933
unbalanced recall 0.16064907370385179
balanced f1 0.5991735897279613
unbalanced f1 0.18514244465670496
balanced auc 0.7536066993967441
unbalanced auc 0.6301918881335544


Interpretation of SVM Results Using LIME

Balanced

In [10]:
from interpret.blackbox import LimeTabular

seed = 1
X, y = balance_dt(minmax_dat, label, seed = seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

pca = PCA()
svc = SVC(gamma='auto', probability=True)

blackbox_model = Pipeline([('pca', pca), ('svc', svc)])
blackbox_model.fit(X_train, y_train)

lime = LimeTabular(predict_fn=blackbox_model.predict_proba, data=X_train)
lime_local = lime.explain_local(X_test[:5], y_test[:5])

show(lime_local)

/Users/nayanayeshlur/opt/anaconda3/lib/python3.8/site-packages/interpret/visual/udash.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/Users/nayanayeshlur/opt/anaconda3/lib/python3.8/site-packages/interpret/visual/udash.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/Users/nayanayeshlur/opt/anaconda3/lib/python3.8/site-packages/interpret/visual/udash.py:7: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


<!-- http://127.0.0.1:7001/140268952080240/ -->

Unbalanced

In [11]:
from interpret.blackbox import LimeTabular

seed = 1
X = minmax_dat
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

pca = PCA()
svc = SVC(gamma='auto', probability=True)

blackbox_model = Pipeline([('pca', pca), ('svc', svc)])
blackbox_model.fit(X_train, y_train)

lime = LimeTabular(predict_fn=blackbox_model.predict_proba, data=X_train)
lime_local = lime.explain_local(X_test[:5], y_test[:5])

show(lime_local)

<!-- http://127.0.0.1:7001/140269023870640/ -->

CNN

In [12]:
best_sc = 0
best_x = []
best_y = []
best_es = None

In [13]:
for i in range(5):
    random.seed(i)
    X, y = balance_dt(minmax_dat, label, seed=i)

    parameters = {'activation': ['relu'], 'solver': ['sgd'],
                  'learning_rate': ['constant'],
                  'hidden_layer_sizes': (90,),
                  'max_iter': [200, 500, 1000], 'random_state': [i]}

    clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1, cv=10, verbose=1, scoring='f1')
    clf.fit(X, y)

    if clf.best_score_ > best_sc:
        best_sc = clf.best_score_
        best_es = clf.best_estimator_
        best_x = X
        best_y = y

Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits


In [14]:
print("-----------------Results--------------------")
print("Best score: ", best_sc)
print(best_es)

-----------------Results--------------------
Best score:  0.3374760420630415
MLPClassifier(hidden_layer_sizes=90, random_state=4, solver='sgd')


In [15]:
eval_with_kfold(best_es, best_x, best_y, minmax_dat, label)

balanced accuracy 0.5895399799263967
unbalanced accuracy 0.7567743373009027
balanced precision 0.6051753262280142


/Users/nayanayeshlur/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/nayanayeshlur/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/nayanayeshlur/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/nayanayeshlur/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_div

unbalanced precision 0.12192982456140351
balanced recall 0.26246834599493535
unbalanced recall 0.021311475409836068
balanced f1 0.3374760420630415
unbalanced f1 0.0228124569381287
balanced auc 0.7413953544265354
unbalanced auc 0.7929447367599616


Interpretation of CNN Results Using Kernel SHAP

Balanced

In [16]:
from interpret.blackbox import ShapKernel
seed = 1
X, y = balance_dt(minmax_dat, label, seed = seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

pca = PCA()
cnn = MLPClassifier(random_state = seed, max_iter=200)

blackbox_model = Pipeline([('pca', pca), ('cnn', cnn)])
blackbox_model.fit(X_train, y_train)

shap = ShapKernel(predict_fn=blackbox_model.predict_proba, data=X_train)
shap_local = shap.explain_local(X_test[:5], y_test[:5])

show(shap_local)

/Users/nayanayeshlur/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

Using 1953 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/5 [00:00<?, ?it/s]

<!-- http://127.0.0.1:7001/140269023965728/ -->

Unbalanced

In [17]:
from interpret.blackbox import ShapKernel
seed = 1
X = minmax_dat
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

pca = PCA()
cnn = MLPClassifier(random_state = seed, max_iter=200)

blackbox_model = Pipeline([('pca', pca), ('cnn', cnn)])
blackbox_model.fit(X_train, y_train)

shap = ShapKernel(predict_fn=blackbox_model.predict_proba, data=X_train)
shap_local = shap.explain_local(X_test[:5], y_test[:5])

show(shap_local)

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
Using 4223 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/5 [00:00<?, ?it/s]

<!-- http://127.0.0.1:7001/140269118508912/ -->

Decision Tree

In [18]:
best_sc = 0
best_x = []
best_y = []
best_es = None
initial_start_time = time.time()

In [19]:
for i in range(5):
    random.seed(i)
    X, y = balance_dt(minmax_dat, label, seed=i)

    parameters = {'criterion': ['gini', 'entropy'],
                  'min_samples_leaf': np.arange(1, 10),
                  'min_samples_split': np.arange(2, 10), 'random_state': [i], }

    clf = GridSearchCV(DecisionTreeClassifier(), parameters, n_jobs=-1, cv=10, verbose=1, scoring='f1')
    clf.fit(X, y)

    if clf.best_score_ > best_sc:
        best_sc = clf.best_score_
        best_es = clf.best_estimator_
        best_x = X
        best_y = y


Fitting 10 folds for each of 144 candidates, totalling 1440 fits
Fitting 10 folds for each of 144 candidates, totalling 1440 fits
Fitting 10 folds for each of 144 candidates, totalling 1440 fits
Fitting 10 folds for each of 144 candidates, totalling 1440 fits
Fitting 10 folds for each of 144 candidates, totalling 1440 fits


In [20]:
print("-----------------Results--------------------")
print("Best score: ", best_sc)
print(best_es)

-----------------Results--------------------
Best score:  0.6255290077927442
DecisionTreeClassifier(min_samples_leaf=9, random_state=4)


In [21]:
eval_with_kfold(best_es, best_x, best_y, minmax_dat, label)

balanced accuracy 0.6502392104382737
unbalanced accuracy 0.7198457535506871
balanced precision 0.6476550161473279
unbalanced precision 0.4931306683568743
balanced recall 0.6140677062508331
unbalanced recall 0.3646208183393309
balanced f1 0.6255290077927442
unbalanced f1 0.3536949511516238
balanced auc 0.6868991201333674
unbalanced auc 0.6460849106808653


Interpreting Decision Tree Using Glassbox Method

Balanced

In [22]:
from interpret.glassbox import ClassificationTree

seed = 1
X, y = balance_dt(minmax_dat, label, seed = seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

dt = ClassificationTree(random_state=seed)
dt.fit(X_train, y_train)

dt_global = dt.explain_global()

show(dt_global)

<!-- http://127.0.0.1:7001/140268601085376/ -->

Unbalanced

In [23]:
from interpret.glassbox import ClassificationTree

seed = 1
X = minmax_dat
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

dt = ClassificationTree(random_state=seed)
dt.fit(X_train, y_train)

dt_global = dt.explain_global()

show(dt_global)

<!-- http://127.0.0.1:7001/140268056667664/ -->

Logistic Regression

In [24]:
best_sc = 0
best_x = []
best_y = []
best_es = None

In [25]:
for i in range(5):
    random.seed(i)
    X, y = balance_dt(minmax_dat, label, seed=i)

    parameters = {'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga'],
                  'random_state': [i], 'max_iter': [100, 300, 500, 1000]}

    clf = GridSearchCV(LogisticRegression(), parameters, n_jobs=-1, cv=10, verbose=1, scoring='recall')
    clf.fit(X, y)

    if clf.best_score_ > best_sc:
        best_sc = clf.best_score_
        best_es = clf.best_estimator_
        best_x = X
        best_y = y

Fitting 10 folds for each of 20 candidates, totalling 200 fits
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Fitting 10 folds for each of 20 candidates, totalling 200 fits


In [26]:
print("-----------------Results--------------------")
print("Best score: ", best_sc)
print(best_es)

-----------------Results--------------------
Best score:  0.3575303212048514
LogisticRegression(random_state=1, solver='liblinear')


In [27]:
eval_with_kfold(best_es, best_x, best_y, minmax_dat, label)

balanced accuracy 0.6284459685513549
unbalanced accuracy 0.7514713069978725
balanced precision 0.666022314096318
unbalanced precision 0.3779363060048535
balanced recall 0.3575303212048514


Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.


unbalanced recall 0.10901639344262293
balanced f1 0.4501521820246272
unbalanced f1 0.13581061277717327
balanced auc 0.7470065525702287
unbalanced auc 0.7858452681689594


Interpreting Logistic Regression Using Glassbox Method

Balanced

In [28]:
from interpret.glassbox import LogisticRegression

seed = 1
X, y = balance_dt(minmax_dat, label, seed = seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

lr = LogisticRegression(random_state = seed)
lr.fit(X_train, y_train)

lr_global = lr.explain_global()
show(lr_global)

<!-- http://127.0.0.1:7001/140268053904256/ -->

Unbalanced

In [29]:
from interpret.glassbox import LogisticRegression

seed = 1
X = minmax_dat
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

lr = LogisticRegression(random_state = seed)
lr.fit(X_train, y_train)

lr_global = lr.explain_global()
show(lr_global)

<!-- http://127.0.0.1:7001/140268059388176/ -->

KNN

In [30]:
best_sc = 0
best_x = []
best_y = []
best_es = None

In [31]:
for i in range(5):
    random.seed(i)
    X, y = balance_dt(minmax_dat, label, seed=i)

    parameters = {'n_neighbors': np.arange(1, 11), 'p': [1, 2]}

    clf = GridSearchCV(KNeighborsClassifier(), parameters, n_jobs=-1, cv=10, verbose=1, scoring='recall')
    clf.fit(X, y)

    if clf.best_score_ > best_sc:
        best_sc = clf.best_score_
        best_es = clf.best_estimator_
        best_x = X
        best_y = y

Fitting 10 folds for each of 20 candidates, totalling 200 fits
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Fitting 10 folds for each of 20 candidates, totalling 200 fits


In [32]:
print("-----------------Results--------------------")
print("Best score: ", best_sc)
print(best_es)

-----------------Results--------------------
Best score:  0.641923230707717
KNeighborsClassifier(n_neighbors=9)


In [33]:
eval_with_kfold(best_es, best_x, best_y, minmax_dat, label)

balanced accuracy 0.6637420541987287
unbalanced accuracy 0.7467371772756024
balanced precision 0.6483665445945961
unbalanced precision 0.6039004449628574
balanced recall 0.641923230707717
unbalanced recall 0.30399840063974415
balanced f1 0.6394443969273291
unbalanced f1 0.3197886957772048
balanced auc 0.7149910527592243
unbalanced auc 0.6906640722859024


Interpreting KNN Using Partial Dependence Plot

Balanced

In [34]:
from interpret.blackbox import PartialDependence
seed = 1
X, y = balance_dt(minmax_dat, label, seed = seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

pca = PCA()
knn = KNeighborsClassifier()

blackbox_model = Pipeline([('pca', pca), ('knn', knn)])
blackbox_model.fit(X_train, y_train)

pdp = PartialDependence(predict_fn=blackbox_model.predict_proba, data=X_train)
pdp_global = pdp.explain_global()

show(pdp_global)

<!-- http://127.0.0.1:7001/140268600972960/ -->

Unbalanced

In [35]:
from interpret.blackbox import PartialDependence
seed = 1
X = minmax_dat
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

pca = PCA()
knn = KNeighborsClassifier()

blackbox_model = Pipeline([('pca', pca), ('knn', knn)])
blackbox_model.fit(X_train, y_train)

pdp = PartialDependence(predict_fn=blackbox_model.predict_proba, data=X_train)
pdp_global = pdp.explain_global()

show(pdp_global)

<!-- http://127.0.0.1:7001/140269090646096/ -->

In [38]:
from interpret.glassbox import ExplainableBoostingClassifier

seed = 1
X, y = balance_dt(minmax_dat, label, seed = seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

ebm = ExplainableBoostingClassifier(random_state=seed)
ebm.fit(X_train, y_train)

ebm_global = ebm.explain_global()
show(ebm_global)

<!-- http://127.0.0.1:7001/140268059597792/ -->

In [39]:
from interpret.glassbox import ExplainableBoostingClassifier

seed = 1
X = minmax_dat
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = seed)

ebm = ExplainableBoostingClassifier(random_state=seed)
ebm.fit(X_train, y_train)

ebm_global = ebm.explain_global()
show(ebm_global)


<!-- http://127.0.0.1:7001/140268063644832/ -->